the best results were achived using the combination of the following parameters:

holdout 0.01 {'loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 3 max_depth: 3 tolerance: 0.001': {'accuracy': 0.14706592313539252, 'precision': 0.14801640190488102, 'recall': 0.14723335299628126, 'f1Score': 0.14060405534462095}}

holdout 0.25 {'loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 4 min_samples_leaf: 1 max_depth: 3 tolerance: 0.001': {'accuracy': 0.14513028207075102, 'precision': 0.14393154878627729, 'recall': 0.14546596893938116, 'f1Score': 0.1391188574071227}}

holdout 0.5 {'loss: log_loss learningRate: 0.1 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 1 max_depth: 3 tolerance: 0.001': {'accuracy': 0.14090850120431192, 'precision': 0.13954088568757558, 'recall': 0.14133017932994482, 'f1Score': 0.1333028517791039}}

holdout 0.75 {'loss: log_loss learningRate: 0.1 n_estimator: 25 min_samples_split: 2 min_samples_leaf: 3 max_depth: 3 tolerance: 0.001': {'accuracy': 0.1309211364693819, 'precision': 0.13103099684433572, 'recall': 0.13146097823017008, 'f1Score': 0.12208886451904581}}

holdout 0.9 {'loss: log_loss learningRate: 0.1 n_estimator: 50 min_samples_split: 4 min_samples_leaf: 1 max_depth: 2 tolerance: 0.001': {'accuracy': 0.12722866780339537, 'precision': 0.12977156730252887, 'recall': 0.1281885037841295, 'f1Score': 0.12224727277625198}}

since the results dont change that much between the various holdouts i decided to take the parameters that are the most frequent and their arounds and use them to train the final model and to use the full train dataset

In [1]:
#parameters
#the parameters here are, at first, the original best value, and then the new values to try
loss = "log_loss" 
learningRates = [0.2, 0.1, 0.05] #0.1 is better for bigger holdouts, adding 0.05 so maybe the results will improve, since i'm using the full dataset 
n_estimators = [50, 75] #added 75 since 50 was the most frequent, maybe the results go up 
min_samples_splits = [2, 3, 4] # 3 out of 5 are with 2, decided to add the middle value
min_samples_leafs = [3, 1, 2] # same reasoning as before
max_depths = [3,  5] # most results are with 3, maybe going higher will improve it 
tolerances = [0.01, 0.001] # all results are with 0.001, the initial parameters where 0.001 and 0.0001, going higher might help

In [2]:
import os
from os import path
import json
import pickle

In [3]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:
import numpy as np
import pandas as pd
import json
import datetime

In [5]:
trainPath = r"C:\Users\alex1\Desktop\DataMining2\dataset\tabular\finalDataset.csv"
testPath = r"C:\Users\alex1\Desktop\DataMining2\dataset\tabular\test.csv"

In [6]:
trainDf = pd.read_csv(trainPath)
testDf = pd.read_csv(testPath)

In [7]:
trainDf.columns

Index(['mode', 'energy', 'acousticness', 'loudness', 'danceability', 'valence',
       'speechiness', 'n_beats', 'time_signature', 'key'],
      dtype='object')

In [8]:
testDf.columns

Index(['mode', 'energy', 'acousticness', 'loudness', 'danceability', 'valence',
       'speechiness', 'n_beats', 'time_signature', 'key'],
      dtype='object')

In [9]:
xTrain = trainDf.drop("key", axis=1)
yTrain = trainDf["key"]

xTest = testDf.drop("key", axis=1)
yTest = testDf["key"]

In [10]:
scl = StandardScaler()
xTrainScaled = scl.fit_transform(xTrain)
xTestScaled = scl.fit_transform(xTest)

In [11]:
xTrainScaled.shape, xTestScaled.shape

((32844, 9), (55982, 9))

In [12]:
results = {}

for learningRate in learningRates:
    for n_estimator in n_estimators:
        for min_samples_split in min_samples_splits:
            for min_samples_leaf in min_samples_leafs:
                for max_depth in max_depths:
                    for tolerance in tolerances:              
                        clf = GradientBoostingClassifier(loss=loss, learning_rate=learningRate, 
                                         n_estimators=n_estimator, min_samples_split=min_samples_split, 
                                         min_samples_leaf=min_samples_leaf, max_depth=max_depth, 
                                         tol=tolerance, random_state=42)

                        key = " ".join(["loss:", str(loss), "learningRate:", str(learningRate), "n_estimator:", str(n_estimator), 
                            "min_samples_split:", str(min_samples_split), "min_samples_leaf:", str(min_samples_leaf), "max_depth:", str(max_depth),
                            "tolerance:", str(tolerance)])
                        
                        print (key)
                        
                        clf.fit(xTrainScaled, yTrain)
                        y_pred = clf.predict(xTestScaled)
                        
                        
                        accuracy = accuracy_score(yTest, y_pred)
                        # Precision
                        precision = precision_score(yTest, y_pred, average='weighted', zero_division=0)
                        # Recall
                        recall = recall_score(yTest, y_pred, average='macro', zero_division=0)
                        # F1 Score
                        f1 = f1_score(yTest, y_pred, average='weighted')
                        
                        results[key] = {
                                "accuracy": accuracy,
                                "precision": precision,
                                "recall": recall,
                                "f1Score": f1,
                            }
                        

loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 3 max_depth: 3 tolerance: 0.01
loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 3 max_depth: 3 tolerance: 0.001
loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 3 max_depth: 5 tolerance: 0.01
loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 3 max_depth: 5 tolerance: 0.001
loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 1 max_depth: 3 tolerance: 0.01
loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 1 max_depth: 3 tolerance: 0.001
loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 1 max_depth: 5 tolerance: 0.01
loss: log_loss learningRate: 0.2 n_estimator: 50 min_samples_split: 2 min_samples_leaf: 1 max_depth: 5 tolerance: 0.001
loss: log_loss learningRate: 0.2 n_estimator

In [13]:
with open (r"C:\Users\alex1\Desktop\DataMining2\src\Classification\Tabular\Gradient Boosting Machines\results on test dataset.json", "w") as f:
    json.dump(results, f, indent = 4)

In [14]:
# Sort the dictionary keys based on the chosen performance metric
sorted_keys = sorted(results.keys(), key=lambda x: (results[x]["accuracy"], results[x]["precision"], results[x]["recall"], results[x]["f1Score"]), reverse=True)

# Create a new dictionary with the ordered keys
ordered_dict = {key: results[key] for key in sorted_keys}

# Save the ordered dictionary to a JSON file
with open (r"C:\Users\alex1\Desktop\DataMining2\src\Classification\Tabular\Gradient Boosting Machines\results on test dataset sorted.json", "w") as f:    
    json.dump(ordered_dict, f, indent=4)